In [13]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import QAOA
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA

# Define the graph for the MaxCut problem
w = np.array([[0, 1, 1, 0, 0],
              [1, 0, 1, 1, 0],
              [1, 1, 0, 1, 0],
              [0, 1, 1, 0, 1],
              [0, 0, 0, 1, 0]])

# Create a Quadratic Program for the MaxCut problem
qp = QuadraticProgram()
n = len(w)
for i in range(n):
    qp.binary_var(name=f"x{i}")

# Set the objective function
for i in range(n):
    for j in range(i):
        if w[i, j] != 0:
            qp.minimize(quadratic={('x'+str(i), 'x'+str(j)): -w[i, j]})

# Convert to QUBO
qp_to_qubo = QuadraticProgramToQubo()
qubo = qp_to_qubo.convert(qp)

# Convert QUBO to PauliSumOp
operator, offset = qubo.to_ising()
# No need to convert to PauliSumOp explicitly, already a SparsePauliOp
# operator = PauliSumOp(operator)

# Parameters for QAOA
p_values = [1, 2, 3, 4, 5]
expectation_qiskit = []

# Qiskit QAOA
backend = Aer.get_backend('aer_simulator')
optimizer = COBYLA(maxiter=100)

for p in p_values:
    qaoa = QAOA(optimizer=optimizer, reps=p, quantum_instance=QuantumInstance(backend, shots=1024))
    result = MinimumEigenOptimizer(qaoa).solve(qubo)
    expectation_qiskit.append(result.fval + offset)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(p_values, expectation_qiskit, label='Qiskit', marker='o')
plt.xlabel('Step number p')
plt.ylabel('Expectation value')
plt.title('QAOA for MaxCut: Qiskit')
plt.legend()
plt.grid(True)
plt.show()

/tmp/ipykernel_145545/3122335358.py:48: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qaoa = QAOA(optimizer=optimizer, reps=p, quantum_instance=QuantumInstance(backend, shots=1024))
/tmp/ipykernel_145545/3122335358.py:48: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(optimizer=optimizer, reps=p, quantum_instance=QuantumInstance(backend, shots=1024))


AttributeError: 'SparsePauliOp' object has no attribute 'primitive_strings'